In [1]:
# Importing required packages
import pandas as pd
import numpy as np
import psycopg2 as pc2
import pandas.io.sql as sqlio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib
import plotly.graph_objects as go
pd.set_option('display.max_rows', None)
import re
from tpot import TPOTRegressor
from dask.distributed import Client
client = Client()
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor

In [2]:
try:
    dbconn = pc2.connect(
        user="admin",
        password="admin",
        host="localhost",
        port="5432",
        database="tmdb")
    dbconn.set_isolation_level(0)
    dbcur = dbconn.cursor()
    sql = "SELECT * FROM movie_full;"
    movie_full = sqlio.read_sql_query(sql, dbconn)
    print("Extracted full table from postgres database.\n")
    dbcur.close()
except (Exception, pc2.Error) as dbError:
    print("Error while connecting to PostgreSQL\n", dbError)
finally:
    print("Proceeding to apply predictive model XGBoost...\n")

Extracted full table from postgres database.

Proceeding to apply predictive model XGBoost...



In [3]:
movie_full.head(5)

,mov_id,title,release_date,status,popularity,runtime,vote_average,vote_count,adult,genre_name,prod_cmpny,prod_cntry,languages,movie_sentiment_score,movie_sentiment_category,review_sentiment_score,review_sentiment_category,percent_profit
0,454626,Sonic the Hedgehog,2020-02-12,Released,272.590,99.0,7.0,410,False,"Action,Science Fiction,Comedy,Family","Original Film,Blur Studios,Marza Animation Pla...","Japan,United States of America",en,7.0,positive,0.000000,neutral,125.668523
1,496243,Parasite,2019-05-30,Released,238.920,132.0,8.6,4419,False,"Comedy,Thriller,Drama","CJ Entertainment,Barunson E&A",South Korea,ko,5.0,positive,33.500000,positive,1669.383420
2,419704,Ad Astra,2019-09-17,Released,160.136,123.0,6.0,2432,False,"Science Fiction,Drama,Thriller,Adventure,Mystery","New Regency Productions,Keep Your Head,MadRive...","Brazil,China,United States of America","en,no",-5.0,negative,23.428571,positive,45.343911
3,495764,Birds of Prey (and the Fantabulous Emancipatio...,2020-02-05,Released,160.411,109.0,6.8,811,False,"Action,Crime,Comedy","DC Entertainment,LuckyChap Entertainment,Kroll...",United States of America,en,-2.0,negative,37.500000,positive,131.234691
4,546554,Knives Out,2019-11-27,Released,129.147,131.0,7.8,2312,False,"Mystery,Thriller,Comedy,Crime,Drama","Lionsgate,FilmNation Entertainment,Ram Bergman...",United States of America,"es,en",-1.0,negative,36.000000,positive,664.833045


In [4]:
movie_full = movie_full.drop(columns = ['mov_id', 'title', 'release_date', 'status'])
movie_full.shape

(4660, 14)

In [5]:
movie_full.describe(include = 'all')

,popularity,runtime,vote_average,vote_count,adult,genre_name,prod_cmpny,prod_cntry,languages,movie_sentiment_score,movie_sentiment_category,review_sentiment_score,review_sentiment_category,percent_profit
count,4660.000000,4660.000000,4660.000000,4660.000000,4660,4660,4644,4659,4660,4660.000000,4660,4660.000000,4660,4.660000e+03
unique,NaN,NaN,NaN,NaN,1,1271,3940,459,600,NaN,3,NaN,3,NaN
top,NaN,NaN,NaN,NaN,False,Drama,Paramount,United States of America,en,NaN,negative,NaN,neutral,NaN
freq,NaN,NaN,NaN,NaN,4660,248,70,2995,3065,NaN,2446,NaN,2514,NaN
mean,12.255583,110.253648,6.494313,1803.071459,NaN,NaN,NaN,NaN,NaN,-0.912017,NaN,5.135243,NaN,3.286500e+04
std,9.468854,20.472851,0.846076,2582.038426,NaN,NaN,NaN,NaN,NaN,5.378723,NaN,12.515398,NaN,1.820755e+06
min,4.631000,26.000000,2.600000,8.000000,NaN,NaN,NaN,NaN,NaN,-25.000000,NaN,-72.500000,NaN,-9.999995e+01
25%,8.410750,96.000000,5.900000,358.000000,NaN,NaN,NaN,NaN,NaN,-4.000000,NaN,0.000000,NaN,3.113490e+00
50%,10.011500,107.000000,6.500000,847.000000,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,0.000000,NaN,1.307890e+02
75%,13.135250,120.000000,7.100000,2068.500000,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,8.000000,NaN,3.576081e+02


In [6]:
movie_full.dtypes

popularity                   float64
runtime                      float64
vote_average                 float64
vote_count                     int64
adult                           bool
genre_name                    object
prod_cmpny                    object
prod_cntry                    object
languages                     object
movie_sentiment_score        float64
movie_sentiment_category      object
review_sentiment_score       float64
review_sentiment_category     object
percent_profit               float64
dtype: object

In [7]:
display(movie_full['genre_name'].value_counts())

Drama                                                                        248
Comedy                                                                       232
Drama,Romance                                                                128
Comedy,Romance                                                               119
Comedy,Drama                                                                  98
Horror,Thriller                                                               91
Comedy,Drama,Romance                                                          89
Horror                                                                        68
Drama,Thriller                                                                54
Action,Thriller                                                               49
Drama,History                                                                 46
Crime,Drama,Thriller                                                          44
Action,Crime,Thriller       

In [8]:
movie_full.loc[(movie_full['genre_name'] == 'Drama,Comedy') | (movie_full['genre_name'] == 'Comedy,Drama'), 'genre_name'] = 'Comedy and Drama'
movie_full.loc[(movie_full['genre_name'].str.contains('Drama')) & (movie_full['genre_name'] != 'Drama') & (movie_full['genre_name'] != 'Comedy and Drama'), 'genre_name'] = 'Drama and Other'
movie_full.loc[(movie_full['genre_name'].str.contains('Comedy')) & (movie_full['genre_name'] != 'Comedy') & (movie_full['genre_name'] != 'Comedy and Drama'), 'genre_name'] = 'Comedy and Other'
movie_full.loc[(movie_full['genre_name'] != 'Drama') & (movie_full['genre_name'] != 'Comedy') & (movie_full['genre_name'] != 'Comedy and Drama') & (movie_full['genre_name'] != 'Comedy and Other') & (movie_full['genre_name'] != 'Drama and Other'), 'genre_name'] = 'Other'
movie_full['genre_name'].value_counts()

Drama and Other     1679
Other               1450
Comedy and Other     916
Drama                248
Comedy               232
Comedy and Drama     135
Name: genre_name, dtype: int64

In [9]:
movie_full['prod_cmpny'].value_counts() # remove from data frame
movie_full = movie_full.drop(columns = ['prod_cmpny'])

In [10]:
movie_full['prod_cntry'].value_counts()

United States of America                                                                                                    2995
United Kingdom,United States of America                                                                                      224
Germany,United States of America                                                                                             107
United Kingdom                                                                                                               107
Canada,United States of America                                                                                               98
France,United States of America                                                                                               46
Australia,United States of America                                                                                            43
France                                                                                           

In [11]:
movie_full.loc[(movie_full['prod_cntry'].str.contains('United States of America')) & (movie_full['prod_cntry'] != 'United States of America'), 'prod_cntry'] = 'USA and Other'
movie_full.loc[(movie_full['prod_cntry'] != 'United States of America') & (movie_full['prod_cntry'] != 'USA and Other'), 'prod_cntry'] = 'Other'
movie_full['prod_cntry'].value_counts()

United States of America    2995
USA and Other               1117
Other                        548
Name: prod_cntry, dtype: int64

In [12]:
movie_full['languages'].value_counts()

en                               3065
en,es                             146
en,fr                              88
es,en                              76
en,ru                              54
fr,en                              46
en,it                              45
de,en                              27
en,de                              27
en,ja                              27
fr                                 26
ja                                 25
es                                 22
ru,en                              17
en,pt                              16
ar,en                              15
en,zh                              15
en,fr,de                           14
it,en                              13
ko                                 13
ru                                 12
zh                                 11
ja,en                              11
en,la                              10
en,vi                              10
en,it,es                           10
en,fr,it    

In [13]:
movie_full.loc[(movie_full['languages'].str.contains('en')) & (movie_full['languages'] != 'en'), 'languages'] = 'en and Other'
movie_full.loc[(movie_full['languages'] != 'en') & (movie_full['languages'] != 'en and Other'), 'languages'] = 'Other'
movie_full['languages'].value_counts()

en              3065
en and Other    1388
Other            207
Name: languages, dtype: int64

In [14]:
movie_full.describe(include = 'all')

,popularity,runtime,vote_average,vote_count,adult,genre_name,prod_cntry,languages,movie_sentiment_score,movie_sentiment_category,review_sentiment_score,review_sentiment_category,percent_profit
count,4660.000000,4660.000000,4660.000000,4660.000000,4660,4660,4660,4660,4660.000000,4660,4660.000000,4660,4.660000e+03
unique,NaN,NaN,NaN,NaN,1,6,3,3,NaN,3,NaN,3,NaN
top,NaN,NaN,NaN,NaN,False,Drama and Other,United States of America,en,NaN,negative,NaN,neutral,NaN
freq,NaN,NaN,NaN,NaN,4660,1679,2995,3065,NaN,2446,NaN,2514,NaN
mean,12.255583,110.253648,6.494313,1803.071459,NaN,NaN,NaN,NaN,-0.912017,NaN,5.135243,NaN,3.286500e+04
std,9.468854,20.472851,0.846076,2582.038426,NaN,NaN,NaN,NaN,5.378723,NaN,12.515398,NaN,1.820755e+06
min,4.631000,26.000000,2.600000,8.000000,NaN,NaN,NaN,NaN,-25.000000,NaN,-72.500000,NaN,-9.999995e+01
25%,8.410750,96.000000,5.900000,358.000000,NaN,NaN,NaN,NaN,-4.000000,NaN,0.000000,NaN,3.113490e+00
50%,10.011500,107.000000,6.500000,847.000000,NaN,NaN,NaN,NaN,-1.000000,NaN,0.000000,NaN,1.307890e+02
75%,13.135250,120.000000,7.100000,2068.500000,NaN,NaN,NaN,NaN,2.000000,NaN,8.000000,NaN,3.576081e+02


In [15]:
movie_full = pd.get_dummies(movie_full)
movie_full.head(5)

,popularity,runtime,vote_average,vote_count,adult,movie_sentiment_score,review_sentiment_score,percent_profit,genre_name_Comedy,genre_name_Comedy and Drama,...,prod_cntry_United States of America,languages_Other,languages_en,languages_en and Other,movie_sentiment_category_negative,movie_sentiment_category_neutral,movie_sentiment_category_positive,review_sentiment_category_negative,review_sentiment_category_neutral,review_sentiment_category_positive
0,272.590,99.0,7.0,410,False,7.0,0.000000,125.668523,0,0,...,0,0,1,0,0,0,1,0,1,0
1,238.920,132.0,8.6,4419,False,5.0,33.500000,1669.383420,0,0,...,0,1,0,0,0,0,1,0,0,1
2,160.136,123.0,6.0,2432,False,-5.0,23.428571,45.343911,0,0,...,0,0,0,1,1,0,0,0,0,1
3,160.411,109.0,6.8,811,False,-2.0,37.500000,131.234691,0,0,...,1,0,1,0,1,0,0,0,0,1
4,129.147,131.0,7.8,2312,False,-1.0,36.000000,664.833045,0,0,...,1,0,0,1,1,0,0,0,0,1


In [16]:
clf = LocalOutlierFactor(n_neighbors=1)
movie_full_a = np.array(movie_full, dtype=np.float)
outliers = clf.fit_predict(movie_full_a)
outliers

array([-1,  1, -1, ...,  1,  1,  1])

In [17]:
movie_full['LOF'] = ""
movie_full['LOF'] = outliers.tolist()
movie_full.shape

(4660, 27)

In [18]:
movie_full = movie_full[movie_full['LOF']==1]
movie_full = movie_full.drop(columns = ['LOF'])
movie_full.shape

(3795, 26)

In [19]:
scaler = StandardScaler()
scaled_movie_full = scaler.fit_transform(movie_full)
scaled_movie_full = pd.DataFrame(scaled_movie_full, index=movie_full.index, columns=movie_full.columns)
movie_profit = np.array(scaled_movie_full['percent_profit'])
scaled_movie_full = scaled_movie_full.drop(columns = ['percent_profit'])
movie_columns = list(scaled_movie_full.columns)
scaled_movie_full = np.array(scaled_movie_full, dtype=np.float)
scaled_movie_full

array([[31.8546156 ,  1.15865333,  2.54004155, ..., -0.30138132,
        -1.12622385,  1.34037688],
       [14.52768389,  0.6426357 ,  2.06156157, ..., -0.30138132,
        -1.12622385,  1.34037688],
       [10.72431712,  0.48783042,  1.82232158, ..., -0.30138132,
        -1.12622385,  1.34037688],
       ...,
       [-0.62659664,  0.2298216 ,  1.22422161, ..., -0.30138132,
         0.88792295, -0.74605883],
       [-0.66194026, -0.38939954, -0.09159834, ..., -0.30138132,
        -1.12622385,  1.34037688],
       [-0.59209453, -0.90541717, -0.68969832, ..., -0.30138132,
         0.88792295, -0.74605883]])

In [20]:
train_movie, test_movie, train_profit, test_profit = train_test_split(scaled_movie_full, movie_profit, test_size = 0.2, random_state = 42)

In [21]:
tpot = TPOTRegressor(verbosity=2, random_state=42, use_dask=True, n_jobs=-1)
tpot.fit(train_movie, train_profit)

Generation 1 - Current best internal CV score: -0.9023833721194944
Generation 2 - Current best internal CV score: -0.8990222397593476
Generation 3 - Current best internal CV score: -0.8990222397593476
Generation 4 - Current best internal CV score: -0.8941928100475571
Generation 5 - Current best internal CV score: -0.8941928100475571
Generation 6 - Current best internal CV score: -0.8941928100475571
Generation 7 - Current best internal CV score: -0.8941928100475571
Generation 8 - Current best internal CV score: -0.8933372931106354
Generation 9 - Current best internal CV score: -0.8933372931106354
Generation 10 - Current best internal CV score: -0.8933372931106354
Generation 11 - Current best internal CV score: -0.8915265087903024
Generation 12 - Current best internal CV score: -0.8915265087903024
Generation 13 - Current best internal CV score: -0.8915265087903024
Generation 14 - Current best internal CV score: -0.8915265087903024
Generation 15 - Current best internal CV score: -0.891526

TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=100,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=-1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=100,
              random_state=42, scoring=None, subsample=1.0, template=None,
              use_dask=True, verbosity=2, warm_start=False)

In [22]:
print(tpot.score(test_movie, test_profit))

-1.3867574566655405


In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_movie)
# Calculate the absolute errors
errors = abs(predictions - test_profit)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'percent profit.')

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_profit)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
rf.score(test_movie, test_profit)